In [1]:
import numpy as np
import matplotlib.pyplot as plt

from scipy import ndimage
import cv2 as cv
import scipy.ndimage
import skimage as skimage

%config InlineBackend.figure_format ='retina'
plt.rcParams.update({'font.size': 12})
from mpl_toolkits.axes_grid1 import make_axes_locatable, axes_size

from tqdm.notebook import tqdm

import sys
# Don't generate the __pycache__ folder locally
sys.dont_write_bytecode = True 
# Print exception without the buit-in python warning
sys.tracebacklimit = 0

In [2]:
from modules import *

In [3]:
FilterKey = 1
LocalSigmaKey = 20
ThresholdValueKey = 20
each_chunk_size = LocalSigmaKey

aspect = 20
pad_fraction = 0.5

SpacingKey = 20 # Spacing between plotting the orientation vectors
ScaleKey = 60 # Length of each vector for plotting

In [4]:
raw_image = convert_to_8bit_grayscale('TestImage1.tif')

############################################################

filtered_image = skimage.filters.gaussian(raw_image, sigma = FilterKey, mode = 'nearest', preserve_range = True)

############################################################

binarized_image = binarize_image(filtered_image)

############################################################

LocalDensityKey = 10

# Define the kernel and it's size
local_kernel_size = LocalDensityKey
if (local_kernel_size % 2 == 0):
    local_kernel_size = local_kernel_size + 1
if (local_kernel_size < 3):
    local_kernel_size = 3

local_kernel = np.ones((local_kernel_size, local_kernel_size), 
                       dtype = np.float32) / (local_kernel_size * local_kernel_size)

############################################################

Local_Density = convolve(binarized_image, local_kernel)

Local_Density = np.divide(Local_Density, Local_Density.max(), 
                          out=np.full(Local_Density.shape, np.nan), 
                          where=Local_Density.max() != 0)

In [5]:
fig = plt.figure(figsize = (8, 6))
im = plt.imshow(Local_Density, vmin = 0, vmax = 1, cmap = 'binary')

plt.title('Local Density')
plt.xticks([])
plt.yticks([])

ax = plt.gca()
divider = make_axes_locatable(ax)
width = axes_size.AxesY(ax, aspect=1./aspect)
pad = axes_size.Fraction(pad_fraction, width)
cax = divider.append_axes("right", size=width, pad=pad)
cbar = plt.colorbar(im, cax=cax)
cbar.formatter.set_powerlimits((0, 0))
# cbar.formatter.set_useMathText(True)
cbar.ax.tick_params(labelsize = 12)

plt.savefig('Local_Density.pdf')
plt.close()

In [6]:
each_chunk_size = int(each_chunk_size)

In [7]:
padded_raw_image = generate_padded_image(raw_image, each_chunk_size)

In [8]:
chunks = split_into_chunks(padded_raw_image, each_chunk_size)

len(chunks)

676

In [9]:
# show_mosaic(chunks, cmap = 'cividis')

In [10]:
print()

Image_Coherance_list = []
Image_Orientation_list = []
vx_list = []
vy_list = []

for i in tqdm(range(len(chunks))):
    
    ##################
    
    current_chunk = chunks[i]

    filtered_chunk = skimage.filters.gaussian(current_chunk, 
                                              sigma = FilterKey, 
                                              mode = 'nearest', 
                                              preserve_range = True)

    # Calculate image gradients in X and Y directions
    image_gradient_x, image_gradient_y = make_image_gradients(filtered_chunk)

    # Calculate the structure tensor and solve for EigenValues, EigenVectors
        
    Structure_Tensor, EigenValues, EigenVectors, Jxx, Jxy, Jyy = make_structure_tensor_2d(image_gradient_x, 
                                                                                          image_gradient_y, 
                                                                                          LocalSigmaKey)

    Image_Coherance = make_coherence(filtered_chunk, EigenValues, Structure_Tensor, ThresholdValueKey)
    
    Image_Orientation = make_orientation(filtered_chunk, Jxx, Jxy, Jyy, ThresholdValueKey)
    vx, vy = make_vxvy(filtered_chunk, EigenVectors, ThresholdValueKey)

    ##################
    
    Image_Coherance_list.append(Image_Coherance)
    Image_Orientation_list.append(Image_Orientation)
    vx_list.append(vx)
    vy_list.append(vy)

print()

  0%|          | 0/676 [00:00<?, ?it/s]

In [11]:
Image_Orientation = stitch_back_chunks(Image_Orientation_list, padded_raw_image, raw_image, each_chunk_size)

In [12]:
Image_Coherance = stitch_back_chunks(Image_Coherance_list, padded_raw_image, raw_image, each_chunk_size)

In [13]:
vx = stitch_back_chunks(vx_list, padded_raw_image, raw_image, each_chunk_size)
vy = stitch_back_chunks(vy_list, padded_raw_image, raw_image, each_chunk_size)

In [14]:
fig = plt.figure(figsize = (8, 6))

# im = plt.imshow(Image_Coherance, vmin = 0, vmax = 1, cmap='Spectral_r')
im = plt.imshow(plt.cm.gray(raw_image/raw_image.max()) * plt.cm.Spectral_r(Image_Coherance), 
            vmin = 0, vmax = 1, cmap = 'Spectral_r')

plt.title('Coherance')

plt.xticks([])
plt.yticks([])
ax = plt.gca()
divider = make_axes_locatable(ax)
width = axes_size.AxesY(ax, aspect=1./aspect)
pad = axes_size.Fraction(pad_fraction, width)
cax = divider.append_axes("right", size=width, pad=pad)
cbar = plt.colorbar(im, cax=cax, cmap='Spectral_r')
cbar.ax.tick_params(labelsize = 12)

plt.savefig('Coherance.pdf')
plt.close()

In [15]:
fig = plt.figure(figsize = (8, 6))

# im = plt.imshow(Image_Orientation, vmin = 0, vmax = 180, cmap='hsv')
im = plt.imshow(plt.cm.gray(raw_image/raw_image.max()) * plt.cm.hsv(Image_Orientation/180), 
           vmin = 0, vmax = 1, cmap = 'hsv')

plt.title('Orientation')

plt.xticks([])
plt.yticks([])
ax = plt.gca()
divider = make_axes_locatable(ax)
width = axes_size.AxesY(ax, aspect=1./aspect)
pad = axes_size.Fraction(pad_fraction, width)
cax = divider.append_axes("right", size=width, pad=pad)
cbar = fig.colorbar(im, cax = cax, ticks = np.linspace(0, 1, 5))
cbar.ax.set_yticklabels([r'$0^{\circ}$', r'$45^{\circ}$', r'$90^{\circ}$', r'$135^{\circ}$', r'$180^{\circ}$'])
ticklabs = cbar.ax.get_yticklabels()
cbar.ax.set_yticklabels(ticklabs, fontsize = 12)

plt.savefig('Orientation.pdf')
plt.close()

In [16]:
fig = plt.figure(figsize = (8, 6))

im = plt.imshow(raw_image, 
                vmin = 0, vmax = 255, 
                cmap = 'Oranges', alpha = 0.8)

plt.title('Local Orientation')

xmesh, ymesh = np.meshgrid(np.arange(raw_image.shape[0]), np.arange(raw_image.shape[1]), indexing = 'ij')

plt.quiver(ymesh[SpacingKey//2::SpacingKey, SpacingKey//2::SpacingKey], xmesh[SpacingKey//2::SpacingKey, SpacingKey//2::SpacingKey], vy[SpacingKey//2::SpacingKey, SpacingKey//2::SpacingKey], vx[SpacingKey//2::SpacingKey, SpacingKey//2::SpacingKey],
scale = ScaleKey, headlength = 0, headaxislength = 0, 
pivot = 'middle', color = 'black', angles = 'xy')

plt.xticks([])
plt.yticks([])

ax = plt.gca()
divider = make_axes_locatable(ax)
width = axes_size.AxesY(ax, aspect=1./aspect)
pad = axes_size.Fraction(pad_fraction, width)
cax = divider.append_axes("right", size=width, pad=pad)
cbar = plt.colorbar(im, cax=cax)
cbar.ax.tick_params(labelsize = 12)

plt.savefig('Local_Orientation.pdf')
plt.close()